# Create Dimension Tables from Cleaned LinkedIn Data
This notebook creates dimension and fact tables from the cleaned LinkedIn job data

In [1]:
import pandas as pd
import re
from tqdm import tqdm

## Load Cleaned Data

In [2]:
# Load the cleaned data from data_cleaning_improved.ipynb output
df = pd.read_excel("../data/processed/linkedin_data_final.xlsx")

print(f"Loaded {len(df)} rows")
print(f"\nColumns: {df.columns.tolist()}")
df.head()

Loaded 15754 rows

Columns: ['job_id', 'title', 'title_clean', 'company', 'place', 'country', 'date', 'description_clean', 'link', 'tools', 'education', 'experience_years', 'experience_level', 'job_type', 'remote_onsite']


,job_id,title,title_clean,company,place,country,date,description_clean,link,tools,education,experience_years,experience_level,job_type,remote_onsite
0,4013873022,Junior Data Analyst,junior data analyst,Trip.com Group,"Amsterdam, North Holland, Netherlands",Netherlands,2024-02-09,About Us Founded in 1999 and listed on NASDAQ ...,https://nl.linkedin.com/jobs/view/junior-data-...,"Excel, SQL, Python, R, Tableau, Power BI, BigQ...",Not Provided,Not Provided,Junior,Internship,Not Provided
1,4019762977,Data Analyst,data analyst,Randstad,"Diemen, North Holland, Netherlands",Netherlands,2024-09-09,Want to get ahead of the game? We are actively...,https://nl.linkedin.com/jobs/view/data-analyst...,Not Provided,Not Provided,Not Provided,Not Provided,Internship,Not Provided
2,4019746811,Data Analyst (Freelance),data analyst (freelance),Swisscom,"Rotterdam, South Holland, Netherlands",Netherlands,2024-09-09,"About the role: As a Data Analyst, you will ha...",https://nl.linkedin.com/jobs/view/data-analyst...,"SQL, Tableau",Not Provided,Not Provided,Not Provided,Freelance,Onsite
3,4032642488,Data Analyst,data analyst,Latitude Amsterdam,"Amsterdam, North Holland, Netherlands",Netherlands,2024-09-25,Are you ready to jump on board with Latitude’s...,https://nl.linkedin.com/jobs/view/data-analyst...,"Excel, SQL, Azure, ETL","Bachelor's, Master's",3,Senior,Not Provided,Not Provided
4,4030973664,Data Analyst,data analyst,Amoria Bond,"Amsterdam, North Holland, Netherlands",Netherlands,2024-09-20,Data Analyst | Dutch Speaking | Freelance | Am...,https://nl.linkedin.com/jobs/view/data-analyst...,SQL,Not Provided,Not Provided,Not Provided,Freelance,Hybrid


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15754 entries, 0 to 15753
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   job_id             15754 non-null  int64         
 1   title              15754 non-null  object        
 2   title_clean        15754 non-null  object        
 3   company            15754 non-null  object        
 4   place              15754 non-null  object        
 5   country            15754 non-null  object        
 6   date               15754 non-null  datetime64[ns]
 7   description_clean  15753 non-null  object        
 8   link               15754 non-null  object        
 9   tools              15754 non-null  object        
 10  education          15754 non-null  object        
 11  experience_years   15754 non-null  object        
 12  experience_level   15754 non-null  object        
 13  job_type           15754 non-null  object        
 14  remote

## Filter Data (Remove 2022 & 2023)

In [4]:
# Add year column if not present
if 'year' not in df.columns:
    df['year'] = pd.to_datetime(df['date']).dt.year
    print("Added 'year' column")

Added 'year' column


In [5]:
# Filter out 2022 and 2023 data
rows_2022 = df[df['year'] == 2022].shape[0]
df = df[df['year'] != 2022]
print(f"✅ Dropped {rows_2022} job listings from 2022.")
print(f"Remaining rows: {df.shape[0]}")

rows_2023 = df[df['year'] == 2023].shape[0]
df = df[df['year'] != 2023]
print(f"✅ Dropped {rows_2023} job listings from 2023.")
print(f"Remaining rows: {df.shape[0]}")

✅ Dropped 1 job listings from 2022.
Remaining rows: 15753
✅ Dropped 3673 job listings from 2023.
Remaining rows: 12080


## Define Skill Categories

In [6]:
# Define skill categories
skill_categories = {
    'soft_skills': [
        "communication", "attention to detail", "problem-solving",
        "presentation", "storytelling", "critical thinking",
        "collaboration", "teamwork", "stakeholder management",
        "curiosity", "innovation"
    ],

    'analytical_skills': [
        "data cleaning", "statistical analysis",
        "exploratory data analysis", "data visualization",
        "data mining", "data exploration",
        "data storytelling", "dashboard"
    ],

    'technical_skills': [
        "sql", "python", "r", "tableau", "power bi", "excel", "looker",
        "power point", "database management",
        "reporting", "documentation", "dax", "macros", "pyspark"
    ],

    'data_engineering': [
        "data warehouse", "data modeling", "apache spark",
        "dbt", "etl", "elt", "data pipelines",
        "airflow", "data lake", "data integration"
    ],

    'ml_skills': [
        "machine learning", "automation", "predictive modeling",
        "clustering", "deep learning", "decision tree"
    ],

    'cloud_skills': [
        "azure", "aws", "databricks", "snowflake", "redshift"
    ],

    'education_level': [
        "bachelor", "master", "phd", "diploma"
    ],

    'degree': [
        "computer science", "engineering", "statistics",
        "mathematics", "economics", "informatics",
        "information systems", "data science"
    ],

    'domain': [
        "financial", "healthcare", "retail",
        "telecommunications", "manufacturing", "technology"
    ]
}

# Define canonical mapping for skill consolidation
CANONICAL_SKILL_MAPPING = {
    "problem solving": "problem-solving",
    "dashboard": "data visualization",
}

print("Skill categories defined")

Skill categories defined


## Helper Functions

In [7]:
# Helper function to extract skills from text
def extract_skills(text, skill_categories):
    found_skills_tuples = set()
    text_lower = str(text).lower()

    for category_name, skills_list in skill_categories.items():
        for skill_keyword in skills_list:
            skill_keyword_lower = skill_keyword.lower()
            
            # Determine canonical name
            canonical_skill_name = CANONICAL_SKILL_MAPPING.get(skill_keyword_lower, skill_keyword_lower)

            # Check if skill is present
            is_found = False
            if skill_keyword_lower == 'r':
                pattern = re.compile(r'\b' + re.escape(skill_keyword_lower) + r'\b')
                if pattern.search(text_lower):
                    is_found = True
            elif skill_keyword_lower == 'problem-solving':
                if 'problem-solving' in text_lower or 'problem solving' in text_lower:
                    is_found = True
            else:
                if skill_keyword_lower in text_lower:
                    is_found = True
            
            if is_found:
                found_skills_tuples.add((canonical_skill_name, category_name))

    return list(found_skills_tuples)

In [8]:
# Helper function to extract seniority level
def extract_seniority(title, description):
    title = str(title).lower()
    description = str(description).lower()

    if any(keyword in title or keyword in description for keyword in ["entry level", "entry-level", "graduate"]):
        return "entry level"
    elif "junior" in title or "junior" in description:
        return "junior"
    elif "mid" in title or "mid-level" in description:
        return "mid-level"
    elif "senior" in title or "senior" in description:
        return "senior"
    elif "lead" in title or "lead" in description:
        return "lead"
    else:
        return "not specified"

## Create Dimension Tables

In [9]:
# dim_company
companies = df[['company']].drop_duplicates().reset_index(drop=True)
companies['company_id'] = companies.index + 1
dim_company = companies[['company_id', 'company']].rename(columns={'company': 'company_name'})

print(f"Created dim_company with {len(dim_company)} companies")
dim_company.head()

Created dim_company with 3816 companies


,company_id,company_name
0,1,Trip.com Group
1,2,Randstad
2,3,Swisscom
3,4,Latitude Amsterdam
4,5,Amoria Bond


In [10]:
# dim_country
dim_country = df[['country']].drop_duplicates().reset_index(drop=True)
dim_country['country_id'] = dim_country.index + 1
dim_country = dim_country[['country_id', 'country']].rename(columns={'country': 'country_name'})

print(f"Created dim_country with {len(dim_country)} countries")
dim_country.head()

Created dim_country with 122 countries


,country_id,country_name
0,1,Netherlands
1,2,Amsterdam Area
2,3,Denmark
3,4,Copenhagen Metropolitan Area
4,5,Canada


In [11]:
# dim_date (UPDATED for Power BI Time Intelligence)
min_date = df['date'].min()
max_date = df['date'].max()

# Create a full date range
all_dates = pd.date_range(start=min_date.floor('D'), end=max_date.ceil('D'))
dim_date = pd.DataFrame({'full_date': all_dates})

# Add various date attributes
dim_date['date_id'] = (dim_date['full_date'] - pd.Timestamp("1899-12-30")).dt.days.astype(int)
dim_date['year'] = dim_date['full_date'].dt.year
dim_date['month_number'] = dim_date['full_date'].dt.month
dim_date['month_name'] = dim_date['full_date'].dt.month_name()
dim_date['day_of_month'] = dim_date['full_date'].dt.day
dim_date['day_of_week_number'] = dim_date['full_date'].dt.dayofweek + 1
dim_date['day_of_week_name'] = dim_date['full_date'].dt.day_name()
dim_date['quarter'] = dim_date['full_date'].dt.quarter
dim_date['quarter_name'] = 'Q' + dim_date['full_date'].dt.quarter.astype(str)
dim_date['week_number'] = dim_date['full_date'].dt.isocalendar().week.astype(int)
dim_date['year_month'] = dim_date['full_date'].dt.strftime('%Y-%m')
dim_date['year_month_name'] = dim_date['full_date'].dt.strftime('%b %Y')
dim_date['year_quarter'] = dim_date['full_date'].dt.year.astype(str) + ' Q' + dim_date['full_date'].dt.quarter.astype(str)

# Select and order columns
dim_date = dim_date[[
    'date_id', 'full_date', 'year', 'month_number', 'month_name',
    'day_of_month', 'day_of_week_number', 'day_of_week_name',
    'quarter', 'quarter_name', 'week_number', 'year_month', 'year_month_name', 'year_quarter'
]]

print(f"Created dim_date with {len(dim_date)} dates")
dim_date.head()

Created dim_date with 700 dates


,date_id,full_date,year,month_number,month_name,day_of_month,day_of_week_number,day_of_week_name,quarter,quarter_name,week_number,year_month,year_month_name,year_quarter
0,45298,2024-01-07,2024,1,January,7,7,Sunday,1,Q1,1,2024-01,Jan 2024,2024 Q1
1,45299,2024-01-08,2024,1,January,8,1,Monday,1,Q1,2,2024-01,Jan 2024,2024 Q1
2,45300,2024-01-09,2024,1,January,9,2,Tuesday,1,Q1,2,2024-01,Jan 2024,2024 Q1
3,45301,2024-01-10,2024,1,January,10,3,Wednesday,1,Q1,2,2024-01,Jan 2024,2024 Q1
4,45302,2024-01-11,2024,1,January,11,4,Thursday,1,Q1,2,2024-01,Jan 2024,2024 Q1


In [12]:
# dim_title
dim_title = df[['title_clean']].drop_duplicates().reset_index(drop=True)
dim_title['title_id'] = dim_title.index + 1
dim_title = dim_title[['title_id', 'title_clean']]

print(f"Created dim_title with {len(dim_title)} unique titles")
dim_title.head()

Created dim_title with 2381 unique titles


,title_id,title_clean
0,1,junior data analyst
1,2,data analyst
2,3,data analyst (freelance)
3,4,commercial data analyst
4,5,senior data analyst


In [13]:
# dim_category
categories_data = [{'category_name': category} for category in skill_categories.keys()]
dim_category = pd.DataFrame(categories_data)
dim_category['category_id'] = dim_category.index + 1

print(f"Created dim_category with {len(dim_category)} categories")
dim_category

Created dim_category with 9 categories


,category_name,category_id
0,soft_skills,1
1,analytical_skills,2
2,technical_skills,3
3,data_engineering,4
4,ml_skills,5
5,cloud_skills,6
6,education_level,7
7,degree,8
8,domain,9


In [14]:
# dim_skill (using canonical skill names)
all_canonical_skills_flat = []
added_canonical_skills = set()

for category_name, skills_list in skill_categories.items():
    category_id = dim_category[dim_category['category_name'] == category_name]['category_id'].iloc[0]
    for skill_keyword in skills_list:
        skill_keyword_lower = skill_keyword.lower()
        
        canonical_skill_name = CANONICAL_SKILL_MAPPING.get(skill_keyword_lower, skill_keyword_lower)

        if skill_keyword_lower == "problem solving":
             canonical_skill_name = "problem-solving"

        if skill_keyword_lower == "dashboard" and canonical_skill_name == "data visualization":
            if ("data visualization", category_id) not in added_canonical_skills:
                all_canonical_skills_flat.append({'skill_name': "data visualization", 'category_id': category_id})
                added_canonical_skills.add(("data visualization", category_id))
        elif (canonical_skill_name, category_id) not in added_canonical_skills:
            all_canonical_skills_flat.append({'skill_name': canonical_skill_name, 'category_id': category_id})
            added_canonical_skills.add((canonical_skill_name, category_id))

dim_skill = pd.DataFrame(all_canonical_skills_flat).reset_index(drop=True)
dim_skill['skill_id'] = dim_skill.index + 1
dim_skill = dim_skill[['skill_id', 'skill_name', 'category_id']]

print(f"Created dim_skill with {len(dim_skill)} unique skills")
dim_skill.head(10)

Created dim_skill with 71 unique skills


,skill_id,skill_name,category_id
0,1,communication,1
1,2,attention to detail,1
2,3,problem-solving,1
3,4,presentation,1
4,5,storytelling,1
5,6,critical thinking,1
6,7,collaboration,1
7,8,teamwork,1
8,9,stakeholder management,1
9,10,curiosity,1


In [15]:
# dim_seniority
seniority_levels = ["entry level", "junior", "mid-level", "senior", "lead", "not specified"]
dim_seniority = pd.DataFrame({'seniority_level': seniority_levels})
dim_seniority['level_id'] = dim_seniority.index + 1
dim_seniority = dim_seniority[['level_id', 'seniority_level']]

print(f"Created dim_seniority with {len(dim_seniority)} levels")
dim_seniority

Created dim_seniority with 6 levels


,level_id,seniority_level
0,1,entry level
1,2,junior
2,3,mid-level
3,4,senior
4,5,lead
5,6,not specified


## Create Fact Table

In [16]:
# Merge with dimension tables to get IDs
fact_jobs = df.merge(dim_company, left_on='company', right_on='company_name', how='left')
fact_jobs = fact_jobs.merge(dim_country, left_on='country', right_on='country_name', how='left')
fact_jobs = fact_jobs.merge(dim_date[['full_date', 'date_id']], left_on='date', right_on='full_date', how='left')
fact_jobs = fact_jobs.merge(dim_title, left_on='title_clean', right_on='title_clean', how='left')

# Select only relevant columns
fact_jobs = fact_jobs[['job_id', 'company_id', 'country_id', 'date_id', 'title_id']]

print(f"Created fact_jobs with {len(fact_jobs)} records")
fact_jobs.head()

Created fact_jobs with 12080 records


,job_id,company_id,country_id,date_id,title_id
0,4013873022,1,1,45331,1
1,4019762977,2,1,45544,2
2,4019746811,3,1,45544,3
3,4032642488,4,1,45560,2
4,4030973664,5,1,45555,2


## Create Bridge Tables

In [17]:
# bridge_job_skill
bridge_job_skill_list = []

# Use description_clean if available, otherwise use description
description_col = 'description_clean' if 'description_clean' in df.columns else 'description'

for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Creating bridge_job_skill"):
    job_id = row['job_id']
    extracted_skills = extract_skills(row[description_col], skill_categories)
    
    for canonical_skill_name, category_name_from_extract in extracted_skills:
        skill_id_series = dim_skill[dim_skill['skill_name'] == canonical_skill_name]['skill_id']
        category_id_series = dim_category[dim_category['category_name'] == category_name_from_extract]['category_id']

        if not skill_id_series.empty and not category_id_series.empty:
            skill_id = skill_id_series.iloc[0]
            category_id = category_id_series.iloc[0]
            bridge_job_skill_list.append({'job_id': job_id, 'skill_id': skill_id, 'category_id': category_id})

bridge_job_skill = pd.DataFrame(bridge_job_skill_list).drop_duplicates()

print(f"\nCreated bridge_job_skill with {len(bridge_job_skill)} records")
bridge_job_skill.head()

Creating bridge_job_skill: 100%|██████████| 12080/12080 [00:23<00:00, 511.09it/s]



Created bridge_job_skill with 72627 records


,job_id,skill_id,category_id
0,4013873022,21,3
1,4013873022,10,1
2,4013873022,7,1
3,4013873022,20,3
4,4013873022,19,3


In [18]:
# bridge_job_seniority
bridge_job_seniority_list = []

for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Creating bridge_job_seniority"):
    job_id = row['job_id']
    
    # Use title_clean and description_clean if available
    title = row.get('title_clean', row.get('title', ''))
    description = row.get('description_clean', row.get('description', ''))
    
    seniority = extract_seniority(title, description)
    level_id_series = dim_seniority[dim_seniority['seniority_level'] == seniority]['level_id']

    if not level_id_series.empty:
        level_id = level_id_series.iloc[0]
        bridge_job_seniority_list.append({'job_id': job_id, 'level_id': level_id})

bridge_job_seniority = pd.DataFrame(bridge_job_seniority_list).drop_duplicates()

print(f"\nCreated bridge_job_seniority with {len(bridge_job_seniority)} records")
bridge_job_seniority.head()

Creating bridge_job_seniority: 100%|██████████| 12080/12080 [00:01<00:00, 7262.11it/s]


Created bridge_job_seniority with 7253 records


,job_id,level_id
0,4013873022,2
1,4019762977,5
2,4019746811,5
3,4032642488,5
4,4030973664,6


## Export All Tables

In [19]:
# Save all tables to Excel
dim_company.to_excel("../model/dim_company.xlsx", index=False)
dim_country.to_excel("../model/dim_country.xlsx", index=False)
dim_date.to_excel("../model/dim_date.xlsx", index=False)
dim_title.to_excel("../model/dim_title.xlsx", index=False)
dim_category.to_excel("../model/dim_category.xlsx", index=False)
dim_skill.to_excel("../model/dim_skill.xlsx", index=False)
dim_seniority.to_excel("../model/dim_seniority.xlsx", index=False)
fact_jobs.to_excel("../model/fact_jobs.xlsx", index=False)
bridge_job_skill.to_excel("../model/bridge_job_skill.xlsx", index=False)
bridge_job_seniority.to_excel("../model/bridge_job_seniority.xlsx", index=False)
print("✅ All dimension, fact, and bridge tables saved successfully!")
print("\n=== Summary ===")
print(f"dim_company: {len(dim_company)} rows")
print(f"dim_country: {len(dim_country)} rows")
print(f"dim_date: {len(dim_date)} rows")
print(f"dim_title: {len(dim_title)} rows")
print(f"dim_category: {len(dim_category)} rows")
print(f"dim_skill: {len(dim_skill)} rows")
print(f"dim_seniority: {len(dim_seniority)} rows")
print(f"fact_jobs: {len(fact_jobs)} rows")
print(f"bridge_job_skill: {len(bridge_job_skill)} rows")
print(f"bridge_job_seniority: {len(bridge_job_seniority)} rows")

✅ All dimension, fact, and bridge tables saved successfully!

=== Summary ===
dim_company: 3816 rows
dim_country: 122 rows
dim_date: 700 rows
dim_title: 2381 rows
dim_category: 9 rows
dim_skill: 71 rows
dim_seniority: 6 rows
fact_jobs: 12080 rows
bridge_job_skill: 72627 rows
bridge_job_seniority: 7253 rows
